In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np 

In [ ]:
# read in labels 
labels = pd.read_csv('clumps_count.csv')

In [ ]:
labels.dropna(inplace=True) 

In [ ]:
labels.head()

In [ ]:
len(labels)

In [ ]:
# split 
train_labels, test_labels = train_test_split(labels, test_size=0.2, random_state=452)
train_labels, val_labels = train_test_split(labels, test_size=1/8, random_state=452)

In [ ]:
train_imgs = train_labels['clump'].values
val_imgs = val_labels['clump'].values 
test_imgs = test_labels['individual'].values 

In [ ]:
# preprocessing 
class SealDataset(Dataset): 
    def __init__(self, img_dir, img_ids, labels, transform_pipeline):
        self.img_dir = img_dir
        self.img_ids = img_ids 
        self.labels = labels 
        self.transform_pipeline = transform_pipeline

    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        row = self.labels.iloc[idx]

        img_path = os.path.join(self.img_dir, f'{self.img_ids[idx]}.tif')
        image = self.transform(Image.open(img_path).convert('RGB'))
        
        label = torch.tensor([
            row['Number of Adult Male Seals'], 
            row['Number of Adult Female Seals'],
            row['Number of Baby Seals']
        ], dtype=torch.float32)

        return image, label

In [ ]:
# transformation pipeline 
transform_pl = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
# loading to class
img_dir = 'clumps' # img dir of clump imgs  

train_set = SealDataset(img_dir, train_imgs, train_labels, transform_pl)
val_set = SealDataset(img_dir, val_imgs, val_labels, transform_pl)
test_set = SealDataset(img_dir, test_imgs, test_labels, transform_pl)

train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [ ]:
# model 
class SealCountRegressor(nn.Module):
    def __init__(self):
        super(SealCountRegressor, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.backbone.fc = nn.Identity()
        self.regressor = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 3),
            nn.Softplus()
        )

    def forward(self, x):
        features = self.backbone(x)
        return self.regressor(features)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = SealCountRegressor()
model = model.to(device) 

In [ ]:
criterion = nn.PoissonNLLLoss(log_input=False) # poisson cuz count data 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# model training 
for epoch in range(10):
    model.train()
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) 

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()*images.size(0)

    print(f"Epoch {epoch+1} - Loss: {running_loss / len(train_loader.dataset):.4f}")

    model.eval()
    val_loss = 0 
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device) 

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
        
        print(f'Val Loss: {val_loss / len(val_loader):.4f}')

In [ ]:
# eval
model.eval()
predicted_males = []
predicted_females = []
predicted_cubs = []

actual_males = []
actual_females = []
actual_cubs = [] 

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)

        outputs = model(images).cpu().squeeze(0)
        labels = labels.squeeze(0)

        predicted_males.append(outputs[0].item())
        predicted_females.append(outputs[1].item())
        predicted_cubs.append(outputs[2].item())

        actual_males.append(labels[0].item())
        actual_females.append(labels[1].item())
        actual_cubs.append(labels[2].item())

predicted_males = np.array(predicted_males)
predicted_females = np.array(predicted_females)
predicted_cubs = np.array(predicted_cubs)
actual_males = np.array(actual_males)
actual_females = np.array(actual_females)
actual_cubs = np.array(actual_cubs)

mae_male = np.mean(np.abs(predicted_males - actual_males))
mae_female = np.mean(np.abs(predicted_females - actual_females))
mae_cubs = np.mean(np.abs(predicted_cubs - actual_cubs))

print(f"MAE - Adult Males:   {mae_male:.2f}")
print(f"MAE - Adult Females: {mae_female:.2f}")
print(f"MAE - Cubs:          {mae_cubs:.2f}")